# Setup enviorment

In [1]:
from Preprocessing.time_series_preprocessing import normalize_train_features, normalize_test_features
from sklearn.model_selection import train_test_split

import tensorflow as tf

import pandas as pd
import os
import numpy as np

2023-08-03 13:43:21.675101: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-03 13:43:21.713952: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-03 13:43:21.715038: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 13:43:22.552611: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### 1. Dataset

In [2]:
""" Read Dataset:"""
embeddings_path = 'Embeddings/variational_autoencoder/variational_autoencoder_ResNet50V2__224_1024_3Bands_no_black_images_full_dataset.csv'
labels_path = '../Tabular_data/dengue_tabular_v2.csv'

""" Dataset Preprocessing:"""
train_percentage = 80
T = 3
autoregressive = False
feature_range = (-1, 1)

In [3]:
embeddings = pd.read_csv(embeddings_path)
embeddings.head()

,Municipality Code,Date,0,1,2,3,4,5,6,7,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,76400,2017-07-30,1.655762,-0.284353,-0.884130,2.468364,-0.369896,-0.799041,-0.822622,-0.016440,...,-1.173675,-0.517362,1.526553,-0.359275,0.024719,0.314871,0.616411,0.979528,0.450631,0.889211
1,76400,2017-11-19,-0.798141,0.156090,0.759308,0.415680,-0.157119,0.175089,0.876964,2.257652,...,0.397869,0.833390,0.051487,-0.307350,-0.545954,-2.025124,0.942197,0.474716,-0.227024,-0.391869
2,76400,2016-11-20,0.036293,-0.695327,-2.338713,-1.637983,-1.029155,-0.467536,-0.599246,2.644862,...,-0.035237,-0.604652,-0.465066,-0.145733,0.163719,-1.455186,1.381717,0.411164,-0.229822,-0.927672
3,76400,2017-07-16,-0.367984,-0.388351,1.078847,0.025843,1.036031,-0.149209,1.250700,1.699442,...,0.152797,0.651429,0.505704,0.368969,0.169281,1.002393,-1.335710,-0.694701,-0.957350,1.159607
4,76400,2017-03-12,-0.733450,-1.864988,-0.011385,-2.051258,-0.319450,-1.088751,-0.932645,0.292711,...,-0.887089,0.462687,-1.128932,1.641547,-0.680204,-0.341273,0.380461,1.057776,-1.272891,0.926245


In [4]:
labels = pd.read_csv(labels_path)
labels.head()

,Municipality code,Municipality,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,Population2014,...,2022/w41,2022/w42,2022/w43,2022/w44,2022/w45,2022/w46,2022/w47,2022/w48,2022/w49,poverty
0,5001,Medellín,2286126,2309689,2331389,2353410,2379920,2410046,2428850,2436071,...,4.0,8.0,6.0,7.0,5.0,10.0,7.0,1.0,0.0,13.749178
1,5002,Abejorral,20643,20944,21197,21355,20973,20067,19732,19796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.079347
2,5004,Abriaquí,2261,2290,2321,2353,2386,2424,2471,2488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.824533
3,5021,Alejandría,3935,4020,4096,4171,4244,4337,4419,4467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.318095
4,5030,Amagá,27996,28331,28624,28839,28776,28371,28259,28432,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.145102


In [5]:
df = pd.merge(labels, embeddings, right_on='Municipality Code', left_on='Municipality code')
df.head()

,Municipality code,Municipality,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,Population2014,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,5045,Apartadó,113220,111883,112089,112914,114241,115856,117088,117992,...,1.131579,0.101524,0.725622,0.245010,-0.015950,-1.647340,-0.335706,-0.759658,0.200150,-0.016401
1,5045,Apartadó,113220,111883,112089,112914,114241,115856,117088,117992,...,-0.685572,0.768649,-1.060369,0.646797,-1.105846,-0.451644,-0.751608,-1.393362,0.333627,-0.147103
2,5045,Apartadó,113220,111883,112089,112914,114241,115856,117088,117992,...,-0.770062,0.191217,-0.341647,-0.208239,-0.449105,-2.136420,-0.823950,0.054495,-0.365410,0.575053
3,5045,Apartadó,113220,111883,112089,112914,114241,115856,117088,117992,...,-0.516224,0.609407,-0.469430,0.564922,0.185102,-0.210481,-0.997317,-0.333545,-0.737093,1.031747
4,5045,Apartadó,113220,111883,112089,112914,114241,115856,117088,117992,...,0.803845,-0.446803,0.520420,0.350235,1.068591,0.644604,0.805475,-0.007781,-1.434304,-0.481188


### Filter Data

In [6]:
columns = ['Municipality Code', 'Municipality', 'Date', 'Population2007', 'Population2008',
           'Population2009', 'Population2010', 'Population2011', 'Population2012',
           'Population2013', 'Population2014', 'Population2015', 'Population2016', 
           'Population2017', 'Population2018', 'Population2019', 'Age0-4(%)', 'Age5-14(%)',
           'Age15-29(%)', 'Age>30(%)', 'AfrocolombianPopulation(%)',
           'IndianPopulation(%)', 'PeoplewithDisabilities(%)',
           'Peoplewhocannotreadorwrite(%)', 'Secondary/HigherEducation(%)',
           'Employedpopulation(%)', 'Unemployedpopulation(%)',
           'Peopledoinghousework(%)', 'Retiredpeople(%)', 'Men(%)', 'Women(%)',
           'Householdswithoutwateraccess(%)', 'Householdswithoutinternetaccess(%)',
           'Buildingstratification1(%)', 'Buildingstratification2(%)',
           'Buildingstratification3(%)', 'Buildingstratification4(%)',
           'Buildingstratification5(%)', 'Buildingstratification6(%)', 
           'NumberofhospitalsperKm2', 'NumberofhousesperKm2', 'poverty'] + list(df.columns[1175:])

# Education and Living Standards
mpi = ['Peoplewhocannotreadorwrite(%)', 'Secondary/HigherEducation(%)',
           'Employedpopulation(%)', 'Unemployedpopulation(%)',
           'Peopledoinghousework(%)', 'Householdswithoutwateraccess(%)', 
           'Householdswithoutinternetaccess(%)',
           'Buildingstratification1(%)', 'Buildingstratification2(%)',
           'Buildingstratification3(%)', 'Buildingstratification4(%)',
           'Buildingstratification5(%)', 'Buildingstratification6(%)', 
           'NumberofhospitalsperKm2', 'NumberofhousesperKm2']

df = df[columns]
df

,Municipality Code,Municipality,Date,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,5045,Apartadó,2017-07-30,113220,111883,112089,112914,114241,115856,117088,...,1.131579,0.101524,0.725622,0.245010,-0.015950,-1.647340,-0.335706,-0.759658,0.200150,-0.016401
1,5045,Apartadó,2016-11-20,113220,111883,112089,112914,114241,115856,117088,...,-0.685572,0.768649,-1.060369,0.646797,-1.105846,-0.451644,-0.751608,-1.393362,0.333627,-0.147103
2,5045,Apartadó,2017-07-16,113220,111883,112089,112914,114241,115856,117088,...,-0.770062,0.191217,-0.341647,-0.208239,-0.449105,-2.136420,-0.823950,0.054495,-0.365410,0.575053
3,5045,Apartadó,2017-02-19,113220,111883,112089,112914,114241,115856,117088,...,-0.516224,0.609407,-0.469430,0.564922,0.185102,-0.210481,-0.997317,-0.333545,-0.737093,1.031747
4,5045,Apartadó,2017-03-12,113220,111883,112089,112914,114241,115856,117088,...,0.803845,-0.446803,0.520420,0.350235,1.068591,0.644604,0.805475,-0.007781,-1.434304,-0.481188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8349,95025,El Retorno,2016-08-14,3300,3387,3517,3703,4491,7035,9114,...,0.559630,0.569307,-1.965172,1.363532,-0.574345,0.217865,0.759732,-0.129125,-1.374886,0.073161
8350,95025,El Retorno,2017-02-26,3300,3387,3517,3703,4491,7035,9114,...,0.195234,0.189281,-0.049411,-0.244062,-0.445049,1.644774,-0.504784,-0.192506,0.320951,1.074259
8351,95025,El Retorno,2018-12-02,3300,3387,3517,3703,4491,7035,9114,...,-0.224778,-1.624294,-0.505348,-0.418190,1.108645,0.864343,-0.187170,0.228194,-0.051805,0.150505
8352,95025,El Retorno,2018-05-20,3300,3387,3517,3703,4491,7035,9114,...,-0.825643,0.647629,0.207195,-0.031073,0.515373,0.229971,-1.150133,-1.101902,-0.603411,-0.722236


### Train Test split

In [7]:
def split_on_column(df, train_percentage, column, random_state=42):
    # Get the unique 'Municipality Code' values
    unique_municipality_codes = pd.unique(df[column])
    
    print(f'Splitting using {column}, using {train_percentage}% for train, and {100-train_percentage}% for test')

    # Split the unique codes into train and test sets
    train_codes, test_codes = train_test_split(unique_municipality_codes, train_size=train_percentage/100, random_state=random_state)

    # Create the train and test sets based on the selected 'Municipality Code' values
    train_data = df[df[column].isin(train_codes)]
    print(f'The resulting train dataset has {train_data.shape[0]} rows, and {train_data.shape[1]} columns')
    test_data = df[df[column].isin(test_codes)]
    print(f'The resulting test dataset has {test_data.shape[0]} rows, and {test_data.shape[1]} columns')
        
    return train_data, test_data

train_df, test_df = split_on_column(df, train_percentage, column='Municipality Code')

Splitting using Municipality Code, using 80% for train, and 20% for test
The resulting train dataset has 6599 rows, and 1065 columns
The resulting test dataset has 1755 rows, and 1065 columns


### Normalize features

In [8]:
# Scale train:
train_df, scalers = normalize_train_features(train_df, feature_range=feature_range)

train_df.head()

,Municipality Code,Municipality,Date,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
103,-1.553536,Barbosa,2017-07-30,-0.581057,-0.580269,-0.580882,-0.58174,-0.582346,-0.583026,-0.583875,...,0.300822,-0.894012,0.235015,0.195948,2.138017,0.650532,-0.294962,0.465480,-0.967125,0.966581
104,-1.553536,Barbosa,2017-11-19,-0.581057,-0.580269,-0.580882,-0.58174,-0.582346,-0.583026,-0.583875,...,-0.493850,-0.514964,1.149728,-0.668418,1.091695,0.643280,1.518980,-0.390376,0.457881,-0.281490
105,-1.553536,Barbosa,2016-11-20,-0.581057,-0.580269,-0.580882,-0.58174,-0.582346,-0.583026,-0.583875,...,0.058121,-0.680784,1.254683,-0.286621,-0.231792,0.174809,0.462033,-0.075391,-0.258419,-0.519269
106,-1.553536,Barbosa,2017-07-16,-0.581057,-0.580269,-0.580882,-0.58174,-0.582346,-0.583026,-0.583875,...,-1.590955,1.177296,-1.039481,0.331760,0.422822,-2.260753,-0.475488,2.314020,0.395654,-1.085908
107,-1.553536,Barbosa,2017-03-12,-0.581057,-0.580269,-0.580882,-0.58174,-0.582346,-0.583026,-0.583875,...,1.341019,-0.618146,1.200579,-0.944087,0.141817,-2.436057,-0.059626,-1.482334,-2.151409,-0.800865


In [9]:
# Scale test:
test_df = normalize_test_features(test_df, scalers=scalers)

test_df.head()

,Municipality Code,Municipality,Date,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-1.55473,Apartadó,2017-07-30,-0.079103,-0.096703,-0.104224,-0.10741,-0.107136,-0.104822,-0.105106,...,1.145377,0.081114,0.733529,0.242734,0.019391,-1.651133,-0.354651,-0.744818,0.235106,-0.024368
1,-1.55473,Apartadó,2016-11-20,-0.079103,-0.096703,-0.104224,-0.10741,-0.107136,-0.104822,-0.105106,...,-0.671244,0.750567,-1.080059,0.643643,-1.065556,-0.448594,-0.772156,-1.376648,0.369946,-0.154023
2,-1.55473,Apartadó,2017-07-16,-0.079103,-0.096703,-0.104224,-0.10741,-0.107136,-0.104822,-0.105106,...,-0.755709,0.171120,-0.350231,-0.209526,-0.411796,-2.143012,-0.844777,0.066927,-0.336232,0.562349
3,-1.55473,Apartadó,2017-02-19,-0.079103,-0.096703,-0.104224,-0.10741,-0.107136,-0.104822,-0.105106,...,-0.501945,0.590769,-0.479989,0.561947,0.219530,-0.206051,-1.018812,-0.319966,-0.711712,1.015385
4,-1.55473,Apartadó,2017-03-12,-0.079103,-0.096703,-0.104224,-0.10741,-0.107136,-0.104822,-0.105106,...,0.817739,-0.469127,0.525157,0.347729,1.099007,0.653927,0.790931,0.004834,-1.416045,-0.485433


# 2. Model

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization

from sklearn.neighbors import KNeighborsRegressor
#from sklearn.linear_model import LinearRegression
#from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

def create_self_attention_model(input_shape):
    query = Input(shape=input_shape)
    value = Dense(128, activation='relu')(query)
    #attention_output = MultiHeadAttention(key_dim=64, num_heads=1)(value, value)
    attention_output = LayerNormalization()(query)
    # Flatten the attention output
    #attention_output_flat = Flatten()(attention_output)
    output = Dense(1)(attention_output)
    model = keras.Model(inputs=query, outputs=output)
    return model


In [11]:
def train_and_evaluate_model(model, train_X, train_y, test_X, test_y):
    if isinstance(model, keras.Sequential):
        # Train Keras model
        model.fit(train_X, train_y, epochs=50, batch_size=32, verbose=0)
        y_pred = model.predict(test_X).flatten()
    else:
        # For other models, follow the previous approach
        model.fit(train_X, train_y)
        y_pred = model.predict(test_X)

    # Calculate evaluation metrics for the current model
    mae = mean_absolute_error(test_y, y_pred)
    mape = mean_absolute_percentage_error(test_y, y_pred)
    smape = 200 * np.mean(np.abs(y_pred - test_y) / (np.abs(y_pred) + np.abs(test_y)))
    rmse = np.sqrt(mean_squared_error(test_y, y_pred))

    return mae, mape, smape, rmse

def evaluate_models(train_X, train_y, test_X, test_y, num_iterations):
    # Create a dictionary to store evaluation results for each model
    model_results = {}

    # Perform iterations
    for i in range(num_iterations):
        print(f"Iteration {i + 1}:")

        # Shuffle the training data for each iteration to have different splits
        indices = np.arange(train_X.shape[0])
        np.random.shuffle(indices)
        train_X_shuffled = train_X[indices]
        train_y_shuffled = train_y[indices]

        # Create the list of regression models, including the Keras model with attention
        models = [
            xgb.XGBRegressor(),
            #KNeighborsRegressor(),
            SVR(),
            create_self_attention_model(input_shape=train_X.shape[1])
        ]

        # Compile the Keras model if it exists in the models list
        for j, model in enumerate(models):
            if isinstance(model, keras.Model):
                model.compile(optimizer='adam', loss='mean_squared_error')
                models[j] = model

        # Train and evaluate models
        for model in models:
            mae, mape, smape, rmse = train_and_evaluate_model(model, train_X_shuffled, train_y_shuffled, test_X, test_y)

            # Append the scores to the lists in the dictionary
            model_name = model.__class__.__name__
            if model_name not in model_results:
                model_results[model_name] = {
                    "mae": [],
                    "mape": [],
                    "smape": [],
                    "rmse": []
                }

            model_results[model_name]["mae"].append(mae)
            model_results[model_name]["mape"].append(mape)
            model_results[model_name]["smape"].append(smape)
            model_results[model_name]["rmse"].append(rmse)

            print(f"{model_name} - MAE: {mae:.2f}, MAPE: {mape:.2f}, SMAPE: {smape:.2f}, RMSE: {rmse:.2f}")

    # Compute the mean and standard deviation for each model and metric
    model_summary = {}
    for model_name, metrics in model_results.items():
        model_summary[model_name] = {
            "mae_mean": np.mean(metrics["mae"]),
            "mae_std": np.std(metrics["mae"]),
            "mape_mean": np.mean(metrics["mape"]),
            "mape_std": np.std(metrics["mape"]),
            "smape_mean": np.mean(metrics["smape"]),
            "smape_std": np.std(metrics["smape"]),
            "rmse_mean": np.mean(metrics["rmse"]),
            "rmse_std": np.std(metrics["rmse"])
        }

    return model_summary

## Exmperiment 1 (Embeddings - Satellite Images)

In [12]:
# Train features and labels set
train_X = train_df.iloc[:, 41:].to_numpy()
train_y = train_df['Secondary/HigherEducation(%)'].to_numpy()

# Test features and labels set
test_X = test_df.iloc[:, 41:].to_numpy()
test_y = test_df['Secondary/HigherEducation(%)'].to_numpy()

In [13]:
# Evaluate the models
evaluate_models(train_X, train_y, test_X, test_y, num_iterations=5)

Iteration 1:
XGBRegressor - MAE: 0.76, MAPE: 0.96, SMAPE: 74.26, RMSE: 1.13
SVR - MAE: 0.95, MAPE: 0.91, SMAPE: 122.06, RMSE: 1.18
55/55 [==============================] - 0s 1ms/step
Functional - MAE: 1.13, MAPE: 1.32, SMAPE: 143.95, RMSE: 1.40
Iteration 2:
XGBRegressor - MAE: 0.76, MAPE: 0.96, SMAPE: 74.26, RMSE: 1.13
SVR - MAE: 0.95, MAPE: 0.91, SMAPE: 122.06, RMSE: 1.18
55/55 [==============================] - 0s 1ms/step
Functional - MAE: 1.17, MAPE: 1.28, SMAPE: 144.62, RMSE: 1.46
Iteration 3:
XGBRegressor - MAE: 0.76, MAPE: 0.96, SMAPE: 74.26, RMSE: 1.13
SVR - MAE: 0.95, MAPE: 0.91, SMAPE: 122.06, RMSE: 1.18
55/55 [==============================] - 0s 1ms/step
Functional - MAE: 1.13, MAPE: 1.29, SMAPE: 142.32, RMSE: 1.43
Iteration 4:
XGBRegressor - MAE: 0.76, MAPE: 0.96, SMAPE: 74.26, RMSE: 1.13
SVR - MAE: 0.95, MAPE: 0.91, SMAPE: 122.06, RMSE: 1.18
55/55 [==============================] - 0s 1ms/step
Functional - MAE: 1.14, MAPE: 1.28, SMAPE: 143.23, RMSE: 1.42
Iteration 5:
XGB

{'XGBRegressor': {'mae_mean': 0.7585212532777815,
  'mae_std': 0.0,
  'mape_mean': 0.9618617841085959,
  'mape_std': 0.0,
  'smape_mean': 74.26458777189022,
  'smape_std': 0.0,
  'rmse_mean': 1.1253954670629307,
  'rmse_std': 0.0},
 'SVR': {'mae_mean': 0.9463159489890914,
  'mae_std': 1.61258647246386e-06,
  'mape_mean': 0.9120887994394259,
  'mape_std': 4.759081577405532e-06,
  'smape_mean': 122.0619977526479,
  'smape_std': 0.0007914939175825835,
  'rmse_mean': 1.1812828749503363,
  'rmse_std': 3.3308851622325453e-07},
 'Functional': {'mae_mean': 1.1477065696938595,
  'mae_std': 0.01717783733619796,
  'mape_mean': 1.290565145667859,
  'mape_std': 0.013027689348968067,
  'smape_mean': 143.79274178257748,
  'smape_std': 0.9267284155069766,
  'rmse_mean': 1.432804324635338,
  'rmse_std': 0.02215587319978666}}

## Exmperiment 2 (Metadata)

In [14]:
train_cols = ['Age0-4(%)', 'Age5-14(%)',
           'Age15-29(%)', 'Age>30(%)', 'AfrocolombianPopulation(%)',
           'IndianPopulation(%)', 'PeoplewithDisabilities(%)',
           'Employedpopulation(%)', 'Unemployedpopulation(%)',
           'Peopledoinghousework(%)', 'Retiredpeople(%)', 'Men(%)', 'Women(%)',
           'Householdswithoutinternetaccess(%)',
           'Buildingstratification1(%)', 'Buildingstratification2(%)',
           'Buildingstratification3(%)', 'Buildingstratification4(%)',
           'Buildingstratification5(%)', 'Buildingstratification6(%)', 
           'NumberofhospitalsperKm2', 'NumberofhousesperKm2']

# Train features and labels set
train_X = train_df[train_cols].to_numpy()
train_y = train_df['Secondary/HigherEducation(%)'].to_numpy()

# Test features and labels set
test_X = test_df[train_cols].to_numpy()
test_y = test_df['Secondary/HigherEducation(%)'].to_numpy()

In [15]:
# Evaluate the models
evaluate_models(train_X, train_y, test_X, test_y, num_iterations=5)

Iteration 1:
XGBRegressor - MAE: 0.33, MAPE: 0.38, SMAPE: 36.61, RMSE: 0.48
SVR - MAE: 0.57, MAPE: 0.43, SMAPE: 66.55, RMSE: 1.01
55/55 [==============================] - 0s 763us/step
Functional - MAE: 0.80, MAPE: 1.44, SMAPE: 134.65, RMSE: 0.97
Iteration 2:
XGBRegressor - MAE: 0.33, MAPE: 0.38, SMAPE: 36.61, RMSE: 0.48
SVR - MAE: 0.57, MAPE: 0.43, SMAPE: 66.55, RMSE: 1.01
55/55 [==============================] - 0s 789us/step
Functional - MAE: 0.98, MAPE: 1.07, SMAPE: 142.22, RMSE: 1.25
Iteration 3:
XGBRegressor - MAE: 0.33, MAPE: 0.38, SMAPE: 36.61, RMSE: 0.48
SVR - MAE: 0.57, MAPE: 0.43, SMAPE: 66.55, RMSE: 1.01
55/55 [==============================] - 0s 772us/step
Functional - MAE: 1.19, MAPE: 1.37, SMAPE: 142.78, RMSE: 1.48
Iteration 4:
XGBRegressor - MAE: 0.33, MAPE: 0.38, SMAPE: 36.61, RMSE: 0.48
SVR - MAE: 0.57, MAPE: 0.43, SMAPE: 66.55, RMSE: 1.01
55/55 [==============================] - 0s 775us/step
Functional - MAE: 1.02, MAPE: 0.97, SMAPE: 133.97, RMSE: 1.45
Iteration 5:

{'XGBRegressor': {'mae_mean': 0.334666647951464,
  'mae_std': 0.0,
  'mape_mean': 0.38255807125730423,
  'mape_std': 0.0,
  'smape_mean': 36.61237399471114,
  'smape_std': 0.0,
  'rmse_mean': 0.48244510707853416,
  'rmse_std': 5.551115123125783e-17},
 'SVR': {'mae_mean': 0.572861728647768,
  'mae_std': 4.965068306494546e-17,
  'mape_mean': 0.4345580599690619,
  'mape_std': 2.4069061620089815e-16,
  'smape_mean': 66.55263623605319,
  'smape_std': 3.240573463185507e-14,
  'rmse_mean': 1.00636940804141,
  'rmse_std': 0.0},
 'Functional': {'mae_mean': 0.9225172716298241,
  'mae_std': 0.19932908681815503,
  'mape_mean': 1.1175288887278365,
  'mape_std': 0.25691762568624726,
  'smape_mean': 137.33894656324378,
  'smape_std': 4.246523033512966,
  'rmse_mean': 1.1870686787348348,
  'rmse_std': 0.2737091030999951}}

### Experiment 3 (Both)

In [16]:
train_cols = ['Age0-4(%)', 'Age5-14(%)',
           'Age15-29(%)', 'Age>30(%)', 'AfrocolombianPopulation(%)',
           'IndianPopulation(%)', 'PeoplewithDisabilities(%)',
           'Employedpopulation(%)', 'Unemployedpopulation(%)',
           'Peopledoinghousework(%)', 'Retiredpeople(%)', 'Men(%)', 'Women(%)',
           'Householdswithoutinternetaccess(%)',
           'Buildingstratification1(%)', 'Buildingstratification2(%)',
           'Buildingstratification3(%)', 'Buildingstratification4(%)',
           'Buildingstratification5(%)', 'Buildingstratification6(%)', 
           'NumberofhospitalsperKm2', 'NumberofhousesperKm2'] + list(train_df.columns[41:])

# Train features and labels set
train_X = train_df[train_cols].to_numpy()
train_y = train_df['Secondary/HigherEducation(%)'].to_numpy()

# Test features and labels set
test_X = test_df[train_cols].to_numpy()
test_y = test_df['Secondary/HigherEducation(%)'].to_numpy()

In [17]:
# Evaluate the models
evaluate_models(train_X, train_y, test_X, test_y, num_iterations=5)

Iteration 1:
XGBRegressor - MAE: 0.32, MAPE: 0.35, SMAPE: 35.28, RMSE: 0.47
SVR - MAE: 0.35, MAPE: 0.37, SMAPE: 47.47, RMSE: 0.45
55/55 [==============================] - 0s 1ms/step
Functional - MAE: 0.56, MAPE: 0.82, SMAPE: 137.03, RMSE: 0.71
Iteration 2:
XGBRegressor - MAE: 0.32, MAPE: 0.35, SMAPE: 35.28, RMSE: 0.47
SVR - MAE: 0.35, MAPE: 0.37, SMAPE: 47.47, RMSE: 0.45
55/55 [==============================] - 0s 1ms/step
Functional - MAE: 0.58, MAPE: 0.88, SMAPE: 138.65, RMSE: 0.73
Iteration 3:
XGBRegressor - MAE: 0.32, MAPE: 0.35, SMAPE: 35.28, RMSE: 0.47
SVR - MAE: 0.35, MAPE: 0.37, SMAPE: 47.47, RMSE: 0.45
55/55 [==============================] - 0s 1ms/step
Functional - MAE: 0.53, MAPE: 0.80, SMAPE: 137.94, RMSE: 0.65
Iteration 4:
XGBRegressor - MAE: 0.32, MAPE: 0.35, SMAPE: 35.28, RMSE: 0.47
SVR - MAE: 0.35, MAPE: 0.37, SMAPE: 47.47, RMSE: 0.45
55/55 [==============================] - 0s 1ms/step
Functional - MAE: 0.54, MAPE: 0.81, SMAPE: 138.08, RMSE: 0.68
Iteration 5:
XGBRegr

{'XGBRegressor': {'mae_mean': 0.3210251429953571,
  'mae_std': 0.0,
  'mape_mean': 0.35427887549465614,
  'mape_std': 0.0,
  'smape_mean': 35.283453213840296,
  'smape_std': 0.0,
  'rmse_mean': 0.47106213250188783,
  'rmse_std': 0.0},
 'SVR': {'mae_mean': 0.34587585074821886,
  'mae_std': 1.5667290153505608e-06,
  'mape_mean': 0.3668525579869771,
  'mape_std': 4.04385049238604e-06,
  'smape_mean': 47.468562143110475,
  'smape_std': 0.0006316507745401182,
  'rmse_mean': 0.45144687228246916,
  'rmse_std': 1.4188032687594945e-06},
 'Functional': {'mae_mean': 0.5532626383032392,
  'mae_std': 0.018712489689900948,
  'mape_mean': 0.8403135046339747,
  'mape_std': 0.03661324602268988,
  'smape_mean': 137.85791726864176,
  'smape_std': 0.5365603393537464,
  'rmse_mean': 0.6939785440877233,
  'rmse_std': 0.025016298561555395}}